In [2]:
import numpy as np
import pandas as pd
import os
import cv2
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
import tensorflow as tf

In [3]:
path_train='../input/einterface-image-dataset/eINTERFACE_2021_Image/train'
path_test='../input/einterface-image-dataset/eINTERFACE_2021_Image/test'
dim=128

In [4]:
x_train=[]
y_train=[]
x_test=[]
y_test=[]
category={'Anger':0,'Disgust':1,'Fear':2,'Happiness':3,'Sadness':4,'Surprise':5}

In [5]:
for i in sorted(os.listdir(path_train)):

    path=os.path.join(path_train,i)
    for j in os.listdir(path):
        path_img=os.path.join(path,j)
        img=cv2.imread(path_img,0) #grayscaling
        img=cv2.resize(img,(dim,dim)) #resizing each image for uniformity
        x_train.append(img)
        y_train.append(category[i])

In [6]:

for i in sorted(os.listdir(path_test)):
  

    path=os.path.join(path_test,i)
    for j in os.listdir(path):
        path_img=os.path.join(path,j)
        img=cv2.imread(path_img,0) #grayscaling
        img=cv2.resize(img,(dim,dim)) #resizing each image for uniformity
        x_test.append(img)
        y_test.append(category[i])


In [12]:
#x_train= np.array(x_train, dtype="float") / 255.0
#x_test=np.array(x_test,dtype="float") / 255.0
x_train_reshaped=X_train_minmax.reshape(X_train_minmax.shape[0],dim,dim,1)
x_test_reshaped=X_test_minmax.reshape(X_test_minmax.shape[0],dim,dim,1)
y_train= np.array(y_train) 
y_test=np.array(y_test)

In [7]:

from sklearn import preprocessing
x=np.array(x_train, dtype="float")
nsamples, nx, ny = x.shape
x_train_dataset = x.reshape((nsamples,nx*ny))
min_max_scaler = preprocessing.MinMaxScaler()
X_train_minmax = min_max_scaler.fit_transform(x_train_dataset)


In [8]:
from sklearn import preprocessing
y=np.array(x_test, dtype="float")
nsamples, nx, ny = y.shape
x_test_dataset = y.reshape((nsamples,nx*ny))
min_max_scaler = preprocessing.MinMaxScaler()
X_test_minmax = min_max_scaler.fit_transform(x_test_dataset)


In [13]:
model=Sequential()

model.add(Conv2D(32, kernel_size = (3,3), input_shape = (128, 128, 1), activation = 'relu'))
model.add(MaxPool2D(pool_size = (2,2)))
model.add(Dropout(0.2))
model.add(Conv2D(32, kernel_size = (3,3), activation = 'relu'))
model.add(MaxPool2D(pool_size = (2,2)))
model.add(Dropout(0.2))
model.add(Conv2D(64, kernel_size = (3,3), activation = 'relu'))
model.add(MaxPool2D(pool_size = (2,2)))
model.add(Dropout(0.2))
model.add(Conv2D(64, kernel_size = (3,3), activation = 'relu'))
model.add(MaxPool2D(pool_size = (2,2)))
model.add(Dropout(0.2))
model.add(Conv2D(128, kernel_size = (3,3), activation = 'relu'))
model.add(MaxPool2D(pool_size = (2,2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(6, activation='softmax'))

In [14]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

checkpoint_filepath = 'best_weights.h'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

model.fit(x_train_reshaped,y_train,validation_split=0.2,batch_size = 100,epochs =150,callbacks=[model_checkpoint_callback],shuffle=True)

In [15]:
model.evaluate(x_test_reshaped,y_test)